In [84]:
# !pip install pandas
# !pip install skikit-learn
# !pip install --upgrade scikit-learn
# !pip install numpy

In [85]:
import pandas as pd
import numpy as np
import requests
from collections import deque

## Declare Target Team and K

In [87]:
target_team = 'Brandon Wheat Kings'
k = 5

## Links and GET functions

In [89]:
key = '41b145a848f4bd67'
def game_id_url_func(num_of_ahead_games, num_of_past_games, current_team_id=''):
  return f'https://lscluster.hockeytech.com/feed/?feed=modulekit&key={key}&view=scorebar&client_code=whl&numberofdaysahead={num_of_ahead_games}&numberofdaysback={num_of_past_games}&season_id=&team_id={current_team_id}&lang_code=en&fmt=json'

def game_stats_url_func(game_id):
  return f'https://lscluster.hockeytech.com/feed/?feed=gc&key={key}&game_id={game_id}&client_code=whl&tab=clock&lang_code=en&fmt=json'

def get_game_ids(url):
  headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "Accept-Encoding": "gzip, deflate, br"  # Request gzip compression
  }

  try:
      response = requests.get(url, headers=headers)

      # Check if the request was successful
      if response.status_code == 200:

          return response.json()['SiteKit']['Scorebar']
          #pprint(response)
      else:
          print(f"Failed to retrieve the page. Status code: {response.status_code}")
  except Exception as e:
      print(f"An error occurred: {e}")


'''
Expects a game_id, which is used to access the Hockey Tech API to get the stats of a game, returned in a JSON format
'''
def get_game_stats(game_id):
  game_stats_url = game_stats_url_func(game_id)
  headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "Accept-Encoding": "gzip, deflate, br"  # Request gzip compression
  }
  
  try:
      response = requests.get(game_stats_url, headers=headers)
  
      # Check if the request was successful
      if response.status_code == 200:
  
          return response.json()
          pprint(response)
      else:
          print(f"Failed to retrieve the page. Status code: {response.status_code}")
  except Exception as e:
      print(f"An error occurred: {e}")

## Import the Dataset

In [91]:
dataset = pd.read_csv('All_teams_WHL_stats.csv')
team_id = {'Spokane Chiefs': '215', 'Seattle Thunderbirds': '214', 'Portland Winterhawks': '208', 'Everett Silvertips': '226', 'Tri-City Americans': '217', 'Kamloops Blazers': '203', 'Kelowna Rockets': '204', 'Prince George Cougars': '210', 'Brandon Wheat Kings': '201', 'Swift Current Broncos': '216', 'Vancouver Giants': '223', 'Victoria Royals': '227', 'Medicine Hat Tigers': '206', 'Edmonton Oil Kings': '228', 'Moose Jaw Warriors': '207', 'Regina Pats': '212', 'Saskatoon Blades': '213', 'Prince Albert Raiders': '209', 'Calgary Hitmen': '202', 'Lethbridge Hurricanes': '205', 'Red Deer Rebels': '211', 'Wenatchee Wild': '222'}

In [92]:
dataset = dataset.sort_values(by="Game_ID")
dataset.reset_index(drop=True, inplace=True)
display(dataset)

,Unnamed: 0,Game_ID,Home_Name,Away_Name,Home_Goals,Away_Goals,Home_PP%,Away_PP%,Home_SOG,Away_SOG,Home_FOW%,Away_FOW%
0,0,1014692,Prince George Cougars,Calgary Hitmen,7,1,0.285714,0.250000,34,17,0.433333,0.566667
1,1,1014699,Kamloops Blazers,Calgary Hitmen,2,4,0.000000,0.000000,32,56,0.492958,0.507042
2,2,1014708,Kelowna Rockets,Calgary Hitmen,3,4,0.166667,0.000000,34,26,0.461538,0.538462
3,3,1014720,Calgary Hitmen,Lethbridge Hurricanes,1,3,0.000000,0.285714,31,37,0.500000,0.500000
4,4,1014735,Red Deer Rebels,Calgary Hitmen,4,2,0.000000,0.000000,31,26,0.576271,0.423729
...,...,...,...,...,...,...,...,...,...,...,...,...
5615,5615,1021563,Victoria Royals,Prince George Cougars,1,2,0.000000,0.000000,38,27,0.470588,0.529412
5616,5616,1021564,Wenatchee Wild,Spokane Chiefs,4,3,0.333333,1.000000,26,29,0.539683,0.460317
5617,5617,1021565,Calgary Hitmen,Medicine Hat Tigers,4,3,0.333333,0.000000,40,25,0.517241,0.482759
5618,5618,1021566,Edmonton Oil Kings,Red Deer Rebels,8,2,0.333333,0.000000,42,22,0.474576,0.525424


## Update the dataset with recent games

In [94]:
# Get the most recent games, and check their game ID
last_game_id = get_game_ids(game_id_url_func(0, 1))
last_game_id = int(last_game_id[-1]["ID"])

# Get the latest game ID we have in the dataset
last_recorded_game_id = dataset.iloc[-1]['Game_ID']

# Get the difference between the last game and the last recorded game.
# this will give us how many games off our dataset is
num_of_game_to_update = last_game_id - last_recorded_game_id
print(f'We need to update {num_of_game_to_update} games in our dataset')

We need to update 0 games in our dataset


In [95]:
update_games = get_game_ids(game_id_url_func(0, num_of_game_to_update))
# print(f'Number of games fetched {len(update_games)}')
# We get a lot of games because the API calls num_of_game_to_update back, and multiple games are played on the same day

In [96]:
update_games = get_game_ids(game_id_url_func(0, num_of_game_to_update))


for game in update_games:
  # Because we got more games than we needed, we ignore the games we already have
  if int(game['ID']) <= last_recorded_game_id:
    continue

  # Get the game stats for each game
  stats = get_game_stats(game["ID"])

  print(stats)
  # Home and visitor team names
  visitor = stats['GC']['Clock']['visiting_team']['name']
  home = stats['GC']['Clock']['home_team']['name']

  # Number of goals
  home_goals = int(stats['GC']['Clock']['home_goal_count'])
  visitor_goals = int(stats['GC']['Clock']['visiting_goal_count'])

  # Calculate Power Play %
  home_ppp_total = float(stats['GC']['Clock']['power_play']['total']['home'])
  visitor_ppp_total = float(stats['GC']['Clock']['power_play']['total']['visiting'])
  # Avoid division by zero for home_ppp
  if home_ppp_total != 0:
      home_ppp = float(stats['GC']['Clock']['power_play']['goals']['home']) / home_ppp_total
  else:
      home_ppp = 0  # Default to 0 if no power plays
  # Avoid division by zero for visitor_ppp
  if visitor_ppp_total != 0:
      visitor_ppp = float(stats['GC']['Clock']['power_play']['goals']['visiting']) / visitor_ppp_total
  else:
      visitor_ppp = 0  # Default to 0 if no power plays

  # Calculate Faceoff Win %
  home_fowp = float(stats['GC']['Clock']['fow']['home'])
  visitor_fowp = float(stats['GC']['Clock']['fow']['visiting'])
  fow_total = home_fowp + visitor_fowp
  if fow_total != 0:
      home_fowp /= fow_total
      visitor_fowp /= fow_total
  else:
      home_fowp, visitor_fowp = .5, .5

  # Shots on goal
  home_sog = sum(stats['GC']['Clock']['shots_on_goal']['home'].values())
  visitor_sog = sum(stats['GC']['Clock']['shots_on_goal']['visiting'].values())
  
  # Add data to the dataset
  dataset.loc[len(dataset)] = {
      "Game_ID": int(game['ID']),
      "Home_Name": home,
      "Away_Name": visitor,
      "Home_Goals": home_goals,
      "Away_Goals": visitor_goals,
      "Home_PP%": home_ppp,
      "Away_PP%": visitor_ppp,
      "Home_SOG": home_sog,
      "Away_SOG": visitor_sog,
      "Home_FOW%": home_fowp,
      "Away_FOW%": visitor_fowp
  }

In [97]:
# Drop Unamed columns
dataset = dataset.loc[:, ~dataset.columns.str.contains('^Unnamed')]
# Update the CSV for future use

display(dataset)

,Unnamed: 0,Game_ID,Home_Name,Away_Name,Home_Goals,Away_Goals,Home_PP%,Away_PP%,Home_SOG,Away_SOG,Home_FOW%,Away_FOW%
0,0,1014692,Prince George Cougars,Calgary Hitmen,7,1,0.285714,0.250000,34,17,0.433333,0.566667
1,1,1014699,Kamloops Blazers,Calgary Hitmen,2,4,0.000000,0.000000,32,56,0.492958,0.507042
2,2,1014708,Kelowna Rockets,Calgary Hitmen,3,4,0.166667,0.000000,34,26,0.461538,0.538462
3,3,1014720,Calgary Hitmen,Lethbridge Hurricanes,1,3,0.000000,0.285714,31,37,0.500000,0.500000
4,4,1014735,Red Deer Rebels,Calgary Hitmen,4,2,0.000000,0.000000,31,26,0.576271,0.423729
...,...,...,...,...,...,...,...,...,...,...,...,...
5615,5615,1021563,Victoria Royals,Prince George Cougars,1,2,0.000000,0.000000,38,27,0.470588,0.529412
5616,5616,1021564,Wenatchee Wild,Spokane Chiefs,4,3,0.333333,1.000000,26,29,0.539683,0.460317
5617,5617,1021565,Calgary Hitmen,Medicine Hat Tigers,4,3,0.333333,0.000000,40,25,0.517241,0.482759
5618,5618,1021566,Edmonton Oil Kings,Red Deer Rebels,8,2,0.333333,0.000000,42,22,0.474576,0.525424


In [98]:
dataset.to_csv('All_teams_WHL_stats.csv')

## Get Next Games

In [100]:
# We want to do a GET request getting the next games played
# Maybe store this information within an [] of (Home Team, Visiting Team)

# Then get a DF and train the model on both teams playing 
num_of_future_days_predict = 1
next_games_response = get_game_ids(game_id_url_func(num_of_future_days_predict, 0))
next_games = []
for game in next_games_response:
  next_games.append((game['HomeLongName'], game["VisitorLongName"]))

print(next_games)

[('Prince Albert Raiders', 'Saskatoon Blades'), ('Swift Current Broncos', 'Brandon Wheat Kings'), ('Medicine Hat Tigers', 'Calgary Hitmen'), ('Red Deer Rebels', 'Lethbridge Hurricanes')]


## Create features and dependencies

In [102]:
# Get default values for averages of goals, PP%, SOG, and FOW%
default_goals = (dataset['Home_Goals'].mean() + dataset['Away_Goals'].mean()) // 2
default_ppp = (dataset['Home_PP%'].mean() + dataset['Away_PP%'].mean()) / 2
default_sog = (dataset['Home_SOG'].mean() + dataset['Away_SOG'].mean()) // 2
default_fowp = (dataset['Home_FOW%'].mean() + dataset['Away_FOW%'].mean()) / 2

print(default_goals, default_ppp, default_sog, default_fowp)

3.0 0.21538569477181574 31.0 0.5


In [103]:
def get_opponent_stats(dataset, queue, opponent_name, row):
  opponent_goals = 0
  opponent_ppp = 0
  opponent_fowp = 0
  opponent_sog = 0
  # Get the last k games averages
  # This creates a new dataset of only opponent games, and which are before the current game
  opponent_games = dataset[
    ((dataset['Home_Name'] == opponent_name) | (dataset['Away_Name'] == opponent_name)) & 
    (dataset['Game_ID'] < row['Game_ID'])
  ].tail(k)

  # If the opponent hasnt played k games yet, use default
  if len(opponent_games) < k:
    opponent_goals = default_goals
    opponent_ppp = default_ppp
    opponent_fowp = default_fowp
    opponent_sog = default_sog
  else:
    # Get the mean of the entire column that the opponent played
    opponent_goals = opponent_games['Home_Goals'].where(opponent_games['Home_Name'] == opponent_name, opponent_games['Away_Goals']).mean()
    opponent_ppp = opponent_games['Home_PP%'].where(opponent_games['Home_Name'] == opponent_name, opponent_games['Away_PP%']).mean()
    opponent_fowp = opponent_games['Home_FOW%'].where(opponent_games['Home_Name'] == opponent_name, opponent_games['Away_FOW%']).mean()
    opponent_sog = opponent_games['Home_SOG'].where(opponent_games['Home_Name'] == opponent_name, opponent_games['Away_SOG']).mean()

  return opponent_goals, opponent_ppp, opponent_fowp, opponent_sog

In [104]:
opponent_win_loss = {}
def generate_team_stats(dataset, team_a, k):
  """
  This function generates a DataFrame with stats comparing a target team (team_a) against its opponent (opponent_team)
  over the last 'k' games. It calculates average stats and differences between the target team and the opponent.

  Parameters:
  - dataset (DataFrame): The dataset containing the game statistics.
  - team_a (str): The name of the target team.
  - k (int): The number of recent games to consider for averaging stats.

  Returns:
  - DataFrame: A DataFrame containing the calculated statistics and differences between the target team and its opponent.
  """

  # Initialize variables and the deque to store the last k games
  target_queue = deque(maxlen=k)
  target_df = pd.DataFrame(columns = [
    "target_goals", 
    "opponent_goals", 
    "target_PP%", 
    "opponent_PP%", 
    "target_SOG", 
    "opponent_SOG",
    "target_FOW%", 
    "opponent_FOW%", 
    "Home/Away",
    # "opponent_win_loss",
    "goals_diff",
    "ppp_diff",
    "sog_diff",
    "fowp_diff",
    "target_win"
  ])

  target_goals = 0
  target_ppp = 0
  target_fowp = 0
  target_sog = 0

  

  # Iterate through the dataset to track the last 'k' games
  for index, row in dataset.iterrows():
    if row['Home_Name'] == team_a or row['Away_Name'] == team_a:

      side = 'Home' if row['Home_Name'] == team_a else 'Away'
      opponent_name = row['Away_Name'] if side == 'Home' else row['Home_Name']
      
      # Update statistics for the target team
      target_goals += row[side + '_Goals']
      target_ppp += row[side + '_PP%']
      target_fowp += row[side + '_FOW%']
      target_sog += row[side + '_SOG']
      
      # If the queue is not full, add the current game to the queue
      if len(target_queue) < k - 1:
        target_queue.append(row.to_dict())
      else:
        # Calculate the opponent's stats and update the win/loss balance
        opponent_goals, opponent_ppp, opponent_fowp, opponent_sog = get_opponent_stats(dataset, target_queue, opponent_name, row)

        target_win = 1 if row[side + '_Goals'] > row[('Away' if side == 'Home' else 'Home') + '_Goals'] else 0
        if opponent_name not in opponent_win_loss:
            opponent_win_loss[opponent_name] = 0.0
        
        # Update opponent win/loss balance
        if target_win:
            opponent_win_loss[opponent_name] -= 1.0
        else:
            opponent_win_loss[opponent_name] += 1.0

        
        # Calculate average stats for the target team
        avg_target_goals = target_goals / k
        avg_target_ppp = target_ppp / k
        avg_target_fowp = target_fowp / k
        avg_target_sog = target_sog / k

        # Calculate differences between the target team and the opponent
        goals_diff = avg_target_goals - opponent_goals
        ppp_diff = avg_target_ppp - opponent_ppp
        sog_diff = avg_target_sog - opponent_sog
        fowp_diff = avg_target_fowp - opponent_fowp

        # 1 is Home, 0 is Away
        target_df.loc[len(target_df)] = {
            "target_goals": avg_target_goals,
            "opponent_goals": opponent_goals,
            "target_PP%": avg_target_ppp,
            "opponent_PP%": opponent_ppp,
            "target_SOG": avg_target_sog,
            "opponent_SOG": opponent_sog,
            "target_FOW%": avg_target_fowp,
            "opponent_FOW%": opponent_fowp,
            "Home/Away": 1.0 if side == 'Home' else 0.0,
            # "opponent_win_loss": opponent_win_loss[opponent_name],
            "goals_diff": goals_diff,
            "ppp_diff": ppp_diff,
            "sog_diff": sog_diff,
            "fowp_diff": fowp_diff,
            "target_win": target_win
        }

        # Remove the oldest game from the queue and update stats
        popped = target_queue.popleft()
        popped_side = 'Home' if popped['Home_Name'] == team_a else 'Away'
        target_goals -= popped[popped_side + '_Goals']
        target_ppp -= popped[popped_side + '_PP%']
        target_fowp -= popped[popped_side + '_FOW%']
        target_sog -= popped[popped_side + '_SOG']

        # Add the current game to the queue
        target_queue.append(row.to_dict())

  return target_df

In [105]:
# loop through the next games, and get the dataframe for each team.
# Probably store it in a [] of (DF Home, DF, Away)
next_games_dfs = []
for game in next_games:
  home_team = game[0]
  away_team = game[1]
  home_team_stats_df = generate_team_stats(dataset, home_team, k)
  away_team_stats_df = generate_team_stats(dataset, away_team, k)
  
  next_games_dfs.append((home_team_stats_df, away_team_stats_df))

a = generate_team_stats(dataset, 'Brandon Wheat Kings', k)
print("K: " + str(k))


K: 5


In [106]:
display(next_games_dfs[1][0])

,target_goals,opponent_goals,target_PP%,opponent_PP%,target_SOG,opponent_SOG,target_FOW%,opponent_FOW%,Home/Away,goals_diff,ppp_diff,sog_diff,fowp_diff,target_win
0,2.8,4.2,0.290476,0.175238,33.8,23.8,0.509522,0.468767,1.0,-1.4,0.115238,10.0,0.040754,1
1,2.2,2.6,0.190476,0.200000,33.6,25.2,0.477458,0.519240,0.0,-0.4,-0.009524,8.4,-0.041782,0
2,3.0,3.0,0.340476,0.215386,34.6,31.0,0.485733,0.500000,0.0,0.0,0.125090,3.6,-0.014267,1
3,2.8,3.0,0.302381,0.215386,32.2,31.0,0.502599,0.500000,1.0,-0.2,0.086995,1.2,0.002599,1
4,3.4,4.6,0.275714,0.240000,27.8,31.2,0.504922,0.483981,1.0,-1.2,0.035714,-3.4,0.020941,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,3.0,3.6,0.173333,0.213810,31.4,31.4,0.506293,0.508342,0.0,-0.6,-0.040476,0.0,-0.002049,0
500,3.0,5.0,0.240000,0.333333,29.2,37.4,0.484465,0.564067,0.0,-2.0,-0.093333,-8.2,-0.079602,0
501,3.4,2.2,0.290000,0.173333,32.4,24.2,0.496908,0.399256,0.0,1.2,0.116667,8.2,0.097652,1
502,3.0,4.8,0.206667,0.333333,31.0,34.8,0.445478,0.527899,0.0,-1.8,-0.126667,-3.8,-0.082421,0


In [107]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# Going to store a dict of the best classifiers between each team
next_games_models = []

for game in next_games_dfs:
  print(len(game))
  
  # Loop through each team (home and away)
  for i, team in enumerate(game):
    best_accuracy = 0
    best_classifier = 0
    best_classifier_name = ""
    
    # Features and dependencies
    X = team.iloc[:, :-1]
    y = team.iloc[:, -1]
    
    # Split into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    
    # Scale train and test sets
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Initialize models
    classifiers = {
        "Random Forest": RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=0),
        "Naive Bayes": GaussianNB(),
        "Logistic Regression": LogisticRegression(random_state=0),
        "SVC": SVC(kernel='rbf', random_state=0)
    }
    
    classifiers_accuracies = {
        "Random Forest": 0,
        "Naive Bayes": 0,
        "Logistic Regression": 0,
        "SVC": 0
    }
    
    # Train each classifier and evaluate accuracy
    for classifier_name, classifier in classifiers.items():
        classifier.fit(X_train, y_train)
        y_pred = classifier.predict(X_test)
        cm = confusion_matrix(y_test, y_pred)
        classifiers_accuracies[classifier_name] = accuracy_score(y_test, y_pred)
        print(f"{classifier_name} accuracy: {accuracy_score(y_test, y_pred)}")
    
    # Get the best classifier for this team
    best_classifier_name = max(classifiers_accuracies, key=classifiers_accuracies.get)
    best_accuracy = classifiers_accuracies[best_classifier_name]
    best_classifier = classifiers[best_classifier_name]
    
    # Append to the models list
    team_name = "Home" if i == 0 else "Away"  # You can label as "Home" or "Away"
    next_games_models.append({
        "team": team_name,
        "classifier": best_classifier,
        "accuracy": best_accuracy,
        "classifier_name": best_classifier_name,
        "scaler": scaler
    })

combine_next_game_models = []
for i in range(0, len(next_games_models), 2):
  home = next_games_models[i]  # Home team entry
  away = next_games_models[i + 1]  # Away team entry

  combine_next_game_models.append((home, away))
  # Now, next_games_models will store the best model and accuracy for both home and away teams in each game.


2
Random Forest accuracy: 0.6435643564356436
Naive Bayes accuracy: 0.5742574257425742
Logistic Regression accuracy: 0.5643564356435643
SVC accuracy: 0.594059405940594
Random Forest accuracy: 0.6153846153846154
Naive Bayes accuracy: 0.5865384615384616
Logistic Regression accuracy: 0.6057692307692307
SVC accuracy: 0.5576923076923077
2
Random Forest accuracy: 0.693069306930693
Naive Bayes accuracy: 0.6831683168316832
Logistic Regression accuracy: 0.6732673267326733
SVC accuracy: 0.7029702970297029
Random Forest accuracy: 0.63
Naive Bayes accuracy: 0.67
Logistic Regression accuracy: 0.65
SVC accuracy: 0.64
2
Random Forest accuracy: 0.6039603960396039
Naive Bayes accuracy: 0.6534653465346535
Logistic Regression accuracy: 0.6435643564356436
SVC accuracy: 0.6336633663366337
Random Forest accuracy: 0.64
Naive Bayes accuracy: 0.59
Logistic Regression accuracy: 0.7
SVC accuracy: 0.74
2
Random Forest accuracy: 0.5882352941176471
Naive Bayes accuracy: 0.6176470588235294
Logistic Regression accurac

In [108]:
print(combine_next_game_models)

[({'team': 'Home', 'classifier': RandomForestClassifier(criterion='entropy', random_state=0), 'accuracy': 0.6435643564356436, 'classifier_name': 'Random Forest', 'scaler': StandardScaler()}, {'team': 'Away', 'classifier': RandomForestClassifier(criterion='entropy', random_state=0), 'accuracy': 0.6153846153846154, 'classifier_name': 'Random Forest', 'scaler': StandardScaler()}), ({'team': 'Home', 'classifier': SVC(random_state=0), 'accuracy': 0.7029702970297029, 'classifier_name': 'SVC', 'scaler': StandardScaler()}, {'team': 'Away', 'classifier': GaussianNB(), 'accuracy': 0.67, 'classifier_name': 'Naive Bayes', 'scaler': StandardScaler()}), ({'team': 'Home', 'classifier': GaussianNB(), 'accuracy': 0.6534653465346535, 'classifier_name': 'Naive Bayes', 'scaler': StandardScaler()}, {'team': 'Away', 'classifier': SVC(random_state=0), 'accuracy': 0.74, 'classifier_name': 'SVC', 'scaler': StandardScaler()}), ({'team': 'Home', 'classifier': LogisticRegression(random_state=0), 'accuracy': 0.696

In [109]:
def get_df_for_predicton(i, next_games_dfs, playing_side):
  side = 0 if playing_side == "Home" else 1
  opponent_side = 0 if side else 1
  target_stats = next_games_dfs[i][side].iloc[-1]
  # print(next_games[i], target_stats)

  opponent_stats = next_games_dfs[i][opponent_side].iloc[-1]
  target_goals = target_stats['target_goals']
  opponent_goals = opponent_stats['target_goals']
  target_PPP = target_stats['target_PP%']
  opponent_PPP = opponent_stats['target_PP%']
  target_SOG = target_stats['target_SOG']
  opponent_SOG = opponent_stats['target_SOG']
  target_FOWP = target_stats['target_FOW%']
  opponent_FOWP = opponent_stats['target_FOW%']
  target_side = opponent_side # This is reverse because I ordered it weirdly
  # print(opponent_win_loss)
  # opponent_win_loss = opponent_win_loss[next_games[i][opponent_side]]
  goals_diff = target_goals - opponent_goals
  ppp_diff = target_PPP - opponent_PPP
  sog_diff = target_SOG - opponent_SOG
  fowp_diff = target_FOWP - opponent_FOWP

  features = [
    [
      target_goals, opponent_goals, target_PPP, opponent_PPP,
      target_SOG, opponent_SOG, target_FOWP, opponent_FOWP,
      side, goals_diff, ppp_diff, sog_diff, fowp_diff
      ]
  ]

  feature_names = ["target_goals", 
      "opponent_goals", 
      "target_PP%", 
      "opponent_PP%", 
      "target_SOG", 
      "opponent_SOG",
      "target_FOW%",
      "opponent_FOW%",
      "Home/Away",
      "goals_diff",
      "ppp_diff",
      "sog_diff",
      "fowp_diff"]


  # Convert features to a DataFrame with the correct column names
  return pd.DataFrame(features, columns=feature_names)
  

In [110]:

for i, game in enumerate(combine_next_game_models):
  # We want to compare the accuracy of the home and away training models
  if game[0]['accuracy'] > game[1]['accuracy']:
    prediction_features_dataframe = get_df_for_predicton(i, next_games_dfs, game[0]["team"])
    side = 0
    # Home is target
  else:
    # Away is target
    prediction_features_dataframe = get_df_for_predicton(i, next_games_dfs, game[1]["team"])
    side = 1

  scaled_prediction_dataframe = game[side]['scaler'].transform(prediction_features_dataframe)
  prediction = game[side]['classifier'].predict(scaled_prediction_dataframe)
  # print(prediction)

  if prediction[0]:
    print(f'{next_games[i][side]} is predicted to with their next game against the {next_games[i][0 if side else 1]} on {next_games_response[i]['GameDate']} with an accuracy of {game[side]['accuracy']:.3f}')
  else:
    print(f'{next_games[i][0 if side else 1]} is predicted to with their next game against the {next_games[i][side]} on {next_games_response[i]['GameDate']} with an accuracy of {game[side]['accuracy']:.3f}')
  # The team with the highest accuracy will be predicted as the target_team
  

Prince Albert Raiders is predicted to with their next game against the Saskatoon Blades on Mon, Dec 30 with an accuracy of 0.644
Brandon Wheat Kings is predicted to with their next game against the Swift Current Broncos on Mon, Dec 30 with an accuracy of 0.703
Calgary Hitmen is predicted to with their next game against the Medicine Hat Tigers on Mon, Dec 30 with an accuracy of 0.740
Lethbridge Hurricanes is predicted to with their next game against the Red Deer Rebels on Mon, Dec 30 with an accuracy of 0.696
